
    <p> <center ><img src="https://mcd.unison.mx/wp-content/themes/awaken/img/logo_mcd.png" width="100" align="center"> </center> </p>

<center>
    <h1>Procesamiento de texto</h1>

    **Ingeniería de Características**

    **MCD/UNISON **

    [Julio Waissman Vilanova](http://mat.uson.mx/~juliowaissman/) (julio.waissman@unison.mx)
</center>


    


# Procesamiento de texto 

## Ingeniería de Características 

### Maestría en Ciencia de Datos 
### Universidad de Sonora



#### Julio Waissman Vilanova (julio.waissman@unison.mx)




El correcto procesamiento de texto es un paso escencial para cualquier tarea de Procesamiento del Lenguaje Natural (PLN). En no pocas ocasiones, la calidad de los resultados se encuentra intimamente relacionada con ésta tarea. Sin embargo, después de la obtención de documentos (una tarea aún más ingrata) esta es una de las tareas menos glamorosa. 

Hay una serie de desiciones que hay que tomar desde esta etapa, y algunas veces es necesario volver a estas instancias para poder obtener un resultado satisfactorio.

## 1. Normalización

### Expresiones regulares en python

La limpieza de texto es un paso crucial, el cual se da a lo largo de todo el procesamiento de texto. Vamos a revisar algunos de los métodos más usuales basados en reglas, los cuales tienen que ver con el manejo de expresiones regulares. Para mayor información sobre expresiones regulares y su uso en *python* se puede consultar el material siguiente:

- Un [*acordeon* del módulo `re` de *python*](https://www.dataquest.io/blog/large_files/python-regular-expressions-cheat-sheet.pdf). 

- Un [tutorial en español sobre el uso de `re`](https://relopezbriega.github.io/blog/2015/07/19/expresiones-regulares-con-python/)

El compilador de expresiones regulares viene con varias banderas de compilación, entre las que destacan `re.I` (para ignorar el uso de mayúsculas y minúsculas); `re.S`, para que el punto signifique cualquier caracter, incluido `\n` (muy práctico en la secuancia `.*` en multiples lineas); `re.M` que permite la búsqueda en múltiples lineas, afectando la operación de los caracteres `^` y `$`; y por último `re.X` para poder representar la expresión regular en forma *verbose* (o verborreica). 

Vamos a hacer algunos ejemplos de expresiones regulares para practicar. Primero, vamos a compilar y explicar algunas fórmulas que típicamente son muy útiles en PLN. 


In [ ]:
import re

email_re = re.compile(r"""
\b               # comienzo de delimitador de palabra
[\w][\w\.-]*     # Cualquier caracter alfanumerico seguido de uno o mas caracteres mas los signos (. -)
@                # seguido de @
\w[\w\.-]*       # cualquier caracter alfanumerico mas los signos (.-)
\.               # seguido de .
[a-zA-Z]{2,6}    # dominio de alto nivel: 2 a 6 letras en minúsculas o mayúsculas.
\b               # fin de delimitador de palabra
""", re.X)

url_re = re.compile(r"""
\b                  # delimitador de palabra
(\w+:\/{2})?        # caracteres iniciales (http, https, ftp, ...) seguidos de // (uno o ninguno)
[\d\w-]+            # cualquier caracter alfanumerico mas -
(\.[\d\w-]+)+       # seguido de uno o mas dominios, que empiezan con punto y siguen con caracteres
(/\S+)*             # cualquier serie de caracteres separados por / y sin espacios en blanco
\b                  # delimitador de palabra
""", re.X) 

insulto_re = re.compile(r"""
([#%&\*\$]{2,})     # al menos dos simbolos típicos para poner insultos
(\w*)               # seguidos de letras
""", re.X)

y a hora vamos a probar algunos de estas expresiones regulares. 

In [ ]:
texto_en_bruto ="""
Como la mayoría sabréis, un sitio Web “normal” tiene una URL en este formato: 
http://www.ordenadores-y-portatiles.com. Habrás notado que www.ordenadores-y-portatiles.com 
sigue llegando sin ningún problema al sitio. que es diferente que
www.ordenadores-y-portatiles.com/una%20rireccion/de%20prueba/pagina.html
Nos podemos encontrar otros formatos como es el caso de FTP, como por ejemplo 
ftp.microsoft.com en modo comando o ftp://ftp.microsoft.com si lo ponemos en un 
navegador de Internet. Tambien se puede accesar por su IP como en  217.76.130.207, 

Para los correos electrónicos, si tienes uno en gmail, lo puede poner como
juliowaissman@gmail.com, JulioWaissman@gmail.com, julio.waissman@gmail.com,
Julio.Waissman@gmail.com, y todos te llevan al mismo lado. Igual se puede tener
correos un poco extraños como w234QWSA.dojdnn_wsda@unison.edu.mx, y deberíamos
poder reconocerlos (entre otros)
"""

procesado, n = re.subn(email_re, ":CORREO:", texto_en_bruto)
procesado, n = re.subn(url_re, ":URL:", procesado)
print("\nY el texto, substituyendo correos electrónicos y url queda como:\n")
print(procesado)

Ahora, vamos a hacer algunas expresiones regulares para detectar emoticones (algo particularmente útil en tratamiento de textos en redes sociales).

In [ ]:
emo_gde_feliz_re = re.compile(r' [8x;:=]-?(?:\)|\}|\]|>){2,}')
emo_chi_feliz_re = re.compile(r' (?:[;:=]-?[\)\}\]d>])|(?:<3)|(?:XOXO)')
emo_gde_triste_re = re.compile(r' [x:=]-?(?:\(|\[|\||\\|/|\{|<){2,}')
emo_chi_triste_re = re.compile(r' [x:=]-?[\(\[\|\\/\{<]')

**Inventa un texto en bruto con diferentes emojis y substituye los emojis por los tokens `:BHAPPY:`, `:SHAPPY:`, `BSAD:`, `:SSAD` segun corresponda.** 

In [ ]:
# ---- Inserta tu código aqui ------

### Tratamiento de texto

Ahora vamos a utilizar las expresiones regulares para tratar un conjunto de documentos, para utilizarlos posteriormente en una tarea de ciencia de datos. Vamos a usar los datos de los tweets del concurso TASS 2015 que se encuentran disponibles para el desarrollo y prueba de sistemas de análisis de sentimientos de manera libre. Para esto, vamos a procesar el corupus de entrenamiento,el cual viene en formato `xml`.

In [ ]:
import xml.etree.ElementTree as et
import pandas as pd

archivo = "general-tweets-train-tagged.xml"

arbol = et.parse(archivo)
raiz = arbol.getroot()
data_dic = []
for tweet in raiz.iter('tweet'):
    contenido = tweet.find('content').text
    if contenido is not None:
        data_dic.append({
            'texto': contenido,
            'polaridad': tweet.find('sentiments')[0].find('value').text,
            'id': tweet.find('tweetid').text,
            'usuario': '@' + tweet.find('user').text,
            'fecha': tweet.find('date').text,
            'tópicos': '[' + ', '.join(['"' + t.text + '"' for t in tweet.find('topics')]) + ']'
        })
df_train = pd.DataFrame.from_dict(data_dic)
display(df_train.head(10))

y vamos a quedarnos sólo con el texto:

In [ ]:
x_train = [documento for documento in df_train['texto'].values]
print('Conjunto con {} entradas de tweeter para ser tratadas'.format(len(x_train)))

x_train[:20]

y el conjunto debera tener 7,218 entradas.

Ahora que ya tenemos los datos de entrada es necesario tratarlos. Para esto, es muy importante que se traten de forma homogenea, y que la forma en que los tratamos sea fácil de exportar, con el fin que sea reproducible. Es por esto que el tratamiento debe de ser siempre realizado en forma de función. Vamos a tratar nuestro texto en una función `prepara_texto`.

In [ ]:
# Vamos a tratar que ver que significa cada expresión regular

usuarios_re = re.compile(r"@[\w\d]+")
hashtags_re = re.compile(r"#[\w\d]+")

remplaza_por_espacios_re = re.compile('[\n/(){}\[\]\|@,;\.]')

simbolos_a_eliminar_re = re.compile('[^\d\w #+_]')

def prepara_texto(texto):
    text = texto.lower()
    
    # Codificaciones (problemas con UTF-8, latin1, etc...)
    text = re.sub(r'\\\\', r'\\', text)
    text = re.sub(r'\\\\', r'\\', text)
    text = re.sub(r'\\x\w{2,2}', ' ', text)
    text = re.sub(r'\\u\w{4,4}', ' ', text)
    text = re.sub(r'\\n', ' . ', text)

    # Cambia e_mails, urls y usuarios por palabra clave
    text = re.sub(email_re, '_EMAIL_', text)
    text = re.sub(url_re, '_URL_', text)
    text = re.sub(usuarios_re, '_USR_', text)
    text = re.sub(hashtags_re, '_HASHTAG_', text)
    
    # Elimina etiquetas de marcaje tipo xml
    # (no se requiere en este caso pero solo para dejar el tip)
    #text = BeautifulSoup(text, "lxml").get_text() 
  
    # Las palabras con letras repetidas más de 3 veces 
    # (dos veces por las personas que abusan demasiado)
    text = re.sub(r'([a-zA-Z])\1\1+(\w*)', r'\1\1\2', text)
    text = re.sub(r'([a-zA-Z])\1\1+(\w*)', r'\1\1\2', text)
    
    # Elimina simbolos
    text = re.sub(remplaza_por_espacios_re, ' ', text)
    text = re.sub(simbolos_a_eliminar_re, '', text)
    
    return text

Ahora vamos a normalizar nuestros documentos

In [ ]:
x_train_prep = [prepara_texto(documento) for documento in x_train]
x_train_prep[:20]

Así como estos ejemplos, existen otros casos en los cuales se pueden establecer reglas, basadas o no en 
expresiones regulares. Algunos casos son:

1. Etiquetas de marcaje (Markdown, $\LaTeX$, ...). 

2. Eliminación de apostrofes

3. Argot y neologísmos

Igualmente, la puntuación puede mantenerse en algunos casos (por ejemplo, los signos de exclamación e interrogación). Por otra parte, usuarios, url, correos electrónicos y demás, pueden ser eliminados en lugar de mantenerlos con una palabra clave (o inclusive, pueden ser mantenidos tal cual, si la base de datos es suficientemente amplia y el nombre del usuario es fundamental para inferir el contexto).
